# Set AWS Session

In [ ]:
# Import necessary libraries
import boto3
import sagemaker

# Get current region, role, and default bucket
aws_region = boto3.Session().region_name  # Get the current AWS region
aws_role = sagemaker.session.Session().get_caller_identity_arn()  # Get the ARN of the caller's identity
output_bucket = sagemaker.Session().default_bucket()  # Get the default S3 bucket for SageMaker output

# Define ANSI escape codes for formatting in the console
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

# Print AWS region, role, and output bucket information
print(f"{bold}aws_region:{unbold} {aws_region}")  # Display AWS region
print(f"{bold}aws_role:{unbold} {aws_role}")  # Display AWS role
print(f"{bold}output_bucket:{unbold} {output_bucket}")  # Display output S3 bucket


# Select Model: Flan T5 XL

In [ ]:
# Import necessary libraries
import IPython
from ipywidgets import Dropdown
from sagemaker.jumpstart.filters import And
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models

# Default model choice
model_id = "huggingface-text2text-flan-t5-xl"  # Set the default model ID

# Identify FLAN T5 models that support fine-tuning using a filter
filter_value = And(
    "task == text2text", "framework == huggingface", "training_supported == true"
)

# List available FLAN T5 models for fine-tuning
model_list = [m for m in list_jumpstart_models(filter=filter_value) if "flan-t5" in m]

# Display the model IDs in a dropdown for user selection
dropdown = Dropdown(
    value=model_id,
    options=model_list,
    description="FLAN T5 models available for fine-tuning:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

# Display a markdown heading and the dropdown for user interaction
display(IPython.display.Markdown("### Select a pre-trained model from the dropdown below"))
display(dropdown)

# Select Instance for Training [ml.p3.16xlarge] & Inference [ml.g5.2xlarge]

In [ ]:
from sagemaker.instance_types import retrieve_default

# Retrieve the selected model ID and set model version
model_id, model_version = dropdown.value, "1.*"

# Retrieve the default instance types for training and inference
training_instance_type = retrieve_default(
    model_id=model_id, model_version=model_version, scope="training"
)
inference_instance_type = retrieve_default(
    model_id=model_id, model_version=model_version, scope="inference"
)

# Display the selected model ID and default instance types
print(f"{bold}model_id:{unbold} {model_id}")  # Display selected model ID
print(f"{bold}training_instance_type:{unbold} {training_instance_type}")  # Display training instance type
print(f"{bold}inference_instance_type:{unbold} {inference_instance_type}")  # Display inference instance type


In [ ]:
# Import necessary modules from SageMaker
from sagemaker import image_uris, model_uris, script_uris

# Training instance will use this image
train_image_uri = image_uris.retrieve(
    region=aws_region,
    framework=None,  # automatically inferred from model_id
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)

# Pre-trained model
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

# Script to execute on the training instance
train_script_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)

# Print the retrieved URIs
print(f"{bold}image uri:{unbold} {train_image_uri}")
print(f"{bold}model uri:{unbold} {train_model_uri}")
print(f"{bold}script uri:{unbold} {train_script_uri}")


# Prepare Traing Data: Get Base Data and Add Prompt

In [ ]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
import numpy as np  # For numerical operations
from io import StringIO  # For handling string input/output operations
import json  # For working with JSON data

In [ ]:
# Set display option to show complete column values without truncation
pd.set_option('display.max_colwidth', None)

In [ ]:
# Initialize Boto3 S3 client
s3_client = boto3.client('s3')

# Specify your S3 bucket name and file key
bucket_name = 'traindatanew'
file_key = 'holcim_fine-tuning_data.csv'  # Get file name

# Get object from S3
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)

# Read the content of the file as a Pandas DataFrame
content = response['Body'].read()
data = pd.read_csv(StringIO(content.decode('utf-8')))


In [ ]:
#prompt
prompt_hanz_sales='''The schema of the table is as below in {} brackets.
              {"table": "hanz_sales",
               "table_description": "It is designed to store daily sales data of different cement materials in New Zealand. The table contains several key columns, including 'Material' for various types of cement material codes, 'Ship-to' for customer codes, 'Material-Description' for detailed material descriptions, and 'Ship-to party' for customer descriptions. Additionally, it tracks important transaction information such as 'Delivery Date' (Del.Date), 'Reported Quantity' (Rpt Qty), and the unit of measurement for quantity, indicated by 'Quantity Unit in Tons' (Reporting UOM). This schema provides essential insights into the sales activities, allowing for a comprehensive analysis of cement material sales in the specified region.",
              "columns": [
                               ["Material": "The various types of cement material type code"],
                               ["Ship-to": "The customer Codes"],
                               ["Material-Description": "Material Description"],
                               ["Ship-to party": "Customer description"],
                               ["Del.Date": "Delivery Date"],
                               ["Rpt Qty": "Reported Quantity]",
                               ["Reporting UOM": "contains value T where Quantity of Unit is Tons"]
                        ]
              }
              Question 1:  What is the total reported quantity for the material '520110'?
              Answer 1: The query should be "SELECT SUM("Rpt Qty") AS total_reported_quantity FROM hanz_sales WHERE "Material" = '520110';"
              Question 2: How many unique ship-to parties are there in the sales data? 
              Answer 2: The query should be "SELECT COUNT(*) AS order_count FROM hanz_sales WHERE "Del.Date" = '2022-01-05';"
              Question 3: What is the total reported quantity for orders delivered in January 2022? 
              Answer 3: SELECT SUM("Rpt Qty") AS total_reported_quantity FROM hanz_sales WHERE strftime('%Y', "Del.Date") = '2022' AND strftime('%m', "Del.Date") = '01'; 
              Question 4: How many orders were delivered for each reporting unit of measure (UOM)?
              Answer 4: SELECT "Reporting UOM", COUNT(*) AS order_count FROM hanz_sales GROUP BY "Reporting UOM"; 
              Question 5: How many orders were delivered for each ship-to party where the delivery date is in January 2022 and the reported quantity is greater than 10 tons? 
              Answer 5: SELECT "Ship-to party", COUNT(*) AS order_count FROM hanz_sales WHERE strftime('%Y', "Del.Date") = '2022' AND strftime('%m', "Del.Date") = '01' AND "Rpt Qty" > 10 GROUP BY "Ship-to party"; 
              Question 6:  What is the total reported quantity for each ship-to party where the reported quantity is less than 1 ton?  
              Answer 6: SELECT "Ship-to party", SUM("Rpt Qty") AS total_reported_quantity FROM hanz_sales WHERE "Rpt Qty" [less than] 1 GROUP BY "Ship-to party"; 
              Question 7: What is the total reported quantity for each ship-to party where the reported quantity is less than and equal to 1 ton?  
              Answer 7: SELECT "Ship-to party", SUM("Rpt Qty") AS total_reported_quantity FROM hanz_sales WHERE "Rpt Qty" [less than equal to] 1 GROUP BY "Ship-to party"; 
              In a Similar way answer generate a SQL query to '''





prompt_au_rmx='''The schema of the table is as below in {} brackets.
              {"table": "au_rmx",
               "table_description": "Table containing billing, customer, material, and sales information, including details such as billing document number, billing date, customer code and name, material description, quantity ordered, plant manufactured, sales value, freight details, account manager, sales office information, and important dates like delivery and order creation.",
              "columns": [
                              ["billing_document": "billing /invoice document number"],
                               ["billing_date": "date when the order was billed"],
                               ["ship_to": "customer code"],
                               ["ship_to_name": "customer name"],
                               ["ship_to_region": "customer region"],
                               ["material": "material brought by the customer]",
                               ["quantity": "quantity ordered"],
                               ["plant": "plant manufactured"],
                               ["plant_name": "plant name"],
                               ["plant_region": "plant region"],
                               ["sales_net_value": "sales value of the material"],
                               ["freight_net_value": "freight value of the material"],
                               ["freight_revenue": "freight revenue of the material"],
                               ["material_description": "material description"],
                               ["cost": "cost of the material"],
                               ["account_manager": "account manager"],
                               ["sales_office": "sales office id"],
                               ["sales_office_desc": "sales office name"],
                               ["del_date": "delivery date"],
                               ["ord_create": "order date"]
                        ]
              }
              Question 1: What is the total sales value for the orders billed on January 6, 2025?
              Answer 1: SELECT SUM(sales_net_value) AS total_sales_value FROM au_rmx WHERE billing_date = '2025-01-06';
              Question 2: Which orders have a delivery date in the year 2025?
              Answer 2: SELECT * FROM au_rmx WHERE strftime('%Y', del_date) = '2025'; 
              Question 3: Which orders have a billing date in January 2025 and a delivery date in February 2025? 
              Answer 3: SELECT * FROM au_rmx WHERE strftime('%Y', billing_date) = '2025' AND strftime('%m', billing_date) = '01' AND strftime('%Y', del_date) = '2025' AND strftime('%m', del_date) = '02'; 
              Question 4: Which sales office had the highest total sales value? 
              Answer 4: SELECT sales_office_desc FROM au_rmx GROUP BY sales_office_desc ORDER BY SUM(sales_net_value) DESC LIMIT 1; 
              Question 5: What is the total quantity of material ordered by each customer? 
              Answer 5: SELECT ship_to_name, SUM(quantity) AS total_quantity_ordered FROM au_rmx GROUP BY ship_to_name;
              Question 6: How many orders have a delivery date before their order date or same as order? 
              Answer 6: SELECT COUNT(*) AS order_count FROM au_rmx WHERE del_date [less than equal to] ord_create;
             Question 7: How many orders have a delivery date before their order date? 
              Answer 7: SELECT COUNT(*) AS order_count FROM au_rmx WHERE del_date [less than] ord_create;
              In a Similar way answer generate a SQL query to '''

In [ ]:
# Add a new column 'context' based on conditions
data['context'] = np.where(
    data['table_name'] == 'hanz_sales',  # Check if 'table_name' is 'hanz_sales'
    prompt_hanz_sales + data['question'],  # If true, concatenate 'prompt_hanz_sales' with 'question'
    np.where(
        data['table_name'] == 'au_rmx',  # Check if 'table_name' is 'au_rmx'
        prompt_au_rmx + data['question'],  # If true, concatenate 'prompt_au_rmx' with 'question'
        None  # If neither condition is true, set 'context' to None
    )
)

# Display the first 10 rows of the modified DataFrame
data.head(10)


In [ ]:
# Splitting the DataFrame into train and test sets with a 35% test size and a random seed for reproducibility
#train, test = train_test_split(data, test_size=0.2, random_state=42)
train, test = train_test_split(data, test_size=0.35, random_state=42)

# Creating copies of the subsets to ensure they are separate DataFrame objects
train = train.copy()  # Copy the train set
# train = train.head(5)  # Optional: Uncomment this line to take only the first 5 rows of the train set
test = test.copy()  # Copy the test set


In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

# write test.csv in S3

In [ ]:
# Create a StringIO buffer to store the CSV data
csv_buffer = StringIO()

# Write the test DataFrame to the CSV buffer, excluding the index column
test.to_csv(csv_buffer, index=False)

# Upload the CSV string to S3
s3_client.put_object(Body=csv_buffer.getvalue(), Bucket=bucket_name, Key='test.csv')

# Write data to csv

In [ ]:
# Create a StringIO buffer to store the CSV data
csv_buffer = StringIO()

# Write the entire DataFrame to the CSV buffer, excluding the index column
data.to_csv(csv_buffer, index=False)

# Upload the CSV string to S3
s3_client.put_object(Body=csv_buffer.getvalue(), Bucket=bucket_name, Key='data.csv')


# Write train.jsonl in S3

In [ ]:
# Import necessary libraries
from sagemaker.s3 import S3Uploader
import json

# Function to convert DataFrame to a list of dictionaries in JSON Lines format
def df_to_jsonl(df, context, sql):
    # Use the apply function to iterate through rows and create dictionaries
    jsonl_data = df.apply(lambda row: json.dumps({"prompt": row[context], "completion": row[sql]}), axis=1).tolist()
    return jsonl_data

# Define column names for context and SQL in the DataFrame
context = 'context'
sql = 'sql'

# Call the function to convert the 'train' DataFrame to a list of dictionaries in JSON Lines format
train_jsonl = df_to_jsonl(train, context, sql)


In [ ]:
# Write the list of dictionaries to a JSON Lines file locally
with open('train.jsonl', 'w') as f:
    # Iterate through the list and write each dictionary as a JSON Lines record to the file
    for item in train_jsonl:
        f.write(f"{str(item)}\n")

# AWS credentials and S3 bucket information
bucket_name = 'your_bucket_name'  # Replace 'your_bucket_name' with the actual S3 bucket name
s3_key = 'train.jsonl'  # Specify the desired path in the S3 bucket

# Create an S3 client (uncomment the following lines if using boto3)
# import boto3
# s3_client = boto3.client('s3', aws_access_key_id='your_access_key', aws_secret_access_key='your_secret_key')

# Upload the local JSON Lines file to S3
# Note: Make sure to uncomment and provide AWS credentials if not already configured
s3_client.upload_file('train.jsonl', bucket_name, s3_key)

In [ ]:
# AWS credentials and S3 bucket information
bucket_name = output_bucket  # Replace 'output_bucket' with your actual S3 bucket name
s3_key = 'train_data/train.jsonl'  # Specify the desired path in the S3 bucket

# Upload the local JSON Lines file to S3
# Note: Uncomment the following line if 's3' is uncommented above and 'train.jsonl' is the local file to be uploaded
s3_client.upload_file('train.jsonl', bucket_name, s3_key)


In [ ]:
# Define the S3 location for training data
train_data_location = f"s3://{output_bucket}/train_data"  # Replace 'output_bucket' with your actual S3 bucket name

# Print the S3 location for reference
print(train_data_location)


In [ ]:
# Import the necessary module for hyperparameters in SageMaker
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
# Note: Replace 'model_id' and 'model_version' with your actual model ID and version
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)


In [ ]:
# Override some default hyperparameters with custom values
hyperparameters["epochs"] = "3"
hyperparameters["max_input_length"] = "1024"  # Data inputs will be truncated at this length
hyperparameters["max_output_length"] = "128"  # Data outputs will be truncated at this length
hyperparameters["batch_size"] = '8'
hyperparameters["learning_rate"] = '0.00001'
hyperparameters["max_grad_norm"] = '5.0'
hyperparameters["peft_type"] = 'lora'
hyperparameters["gradient_accumulation_steps"] = '4'

# Print the updated hyperparameters for reference
print(hyperparameters)


In [ ]:
# Import necessary modules for SageMaker Estimator
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base

# Extract the most informative part of the model ID for better readability
model_name = "-".join(model_id.split("-")[2:])

# Generate a training job name based on the model name and hyperparameter settings
training_job_name = name_from_base(f"js-demo-{model_name}-{hyperparameters['epochs']}")

# Print the generated training job name for reference
print(f"{bold}job name:{unbold} {training_job_name}")


In [ ]:
output_location='s3://finetune-model/bycode/'

In [ ]:
# Define the training metric definitions to capture key metrics during training.
training_metric_definitions = [
    {"Name": "val_loss", "Regex": "'eval_loss': ([0-9\\.]+)"},
    {"Name": "train_loss", "Regex": "'loss': ([0-9\\.]+)"},
    {"Name": "epoch", "Regex": "'epoch': ([0-9\\.]+)"},
]

# Create SageMaker Estimator instance with specified configurations.
sm_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    model_uri=train_model_uri,
    source_dir=train_script_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    volume_size=300,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=output_location,
    metric_definitions=training_metric_definitions,  # Attach metric definitions for tracking training metrics.
)


In [ ]:
# Launch a SageMaker training job over data located in the given S3 path
# Training jobs can take hours, it is recommended to set wait=False,
# and monitor job status through SageMaker console
sm_estimator.fit({"training": train_data_location}, job_name=training_job_name, wait=False)

In [ ]:
from sagemaker import TrainingJobAnalytics

# This can be called while the job is still running
df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()
df

In [ ]:
training_job_name

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

# Generate a name for the fine-tuned model based on the model_id
fine_tuned_name = name_from_base(f"jumpstart-demo-fine-tuned-{model_id}")

# Construct the URI for the fine-tuned model using the output location and training job name
fine_tuned_model_uri = f"{output_location}{training_job_name}/output/model.tar.gz"

In [ ]:
fine_tuned_name

In [ ]:
fine_tuned_model_uri

In [ ]:
from sagemaker import image_uris

# Retrieve the inference docker image URI using SageMaker image_uris module.
# The base HuggingFace container image is automatically inferred from the provided model_id.
# The image is scoped for inference, and the instance type for deployment is specified.
deploy_image_uri = image_uris.retrieve(
    region=None,  # Region is automatically inferred from the SageMaker session
    framework=None,  # Framework is automatically inferred from model_id
    model_id=model_id,
    model_version=model_version,
    image_scope="inference",  # Set the image scope for inference
    instance_type=inference_instance_type,  # Specify the instance type for deployment
)


In [ ]:
# Create the SageMaker model instance for the fine-tuned model.
# The model is configured with the deployment image URI, fine-tuned model URI,
# AWS role, custom predictor class, and a specified name.
fine_tuned_model = Model(
    image_uri=deploy_image_uri,
    model_data=fine_tuned_model_uri,
    role=aws_role,
    predictor_cls=Predictor,  # Custom predictor class for the model
    name=fine_tuned_name,  # Name for the SageMaker model instance
)

# Print the image URI and model URI for reference.
print(f"{bold}image URI:{unbold}{newline} {deploy_image_uri}")
print(f"{bold}model URI:{unbold}{newline} {fine_tuned_model_uri}")

# Indicate the start of the deployment process.
print("Deploying an endpoint ...")


In [ ]:
# Deploy the fine-tuned model using the SageMaker deploy method.
# The deployment is configured with an initial instance count, instance type,
# custom predictor class, and a specified endpoint name.
fine_tuned_predictor = fine_tuned_model.deploy(
    initial_instance_count=1,  # Number of instances to deploy initially
    instance_type=inference_instance_type,  # Type of instance for deployment
    predictor_cls=Predictor,  # Custom predictor class for the deployed model
    endpoint_name=fine_tuned_name,  # Specify the name for the deployed endpoint
)

# Print a message indicating the successful deployment.
print(f"{newline}Deployed an endpoint {fine_tuned_name}")


# Inference

In [ ]:
# Initialize Boto3 S3 client
s3_client = boto3.client('s3')

# Specify your S3 bucket name and file key
bucket_name = 'sagemaker-ap-southeast-1-789372484921'
# Get file name
file_key = 'data.csv'

# Get object from S3
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)

# Read the content of the file as a Pandas DataFrame
content = response['Body'].read()
read_data = pd.read_csv(StringIO(content.decode('utf-8')))


In [ ]:
print(read_data.shape)
read_data.head()

In [ ]:
import boto3
import json

# Parameters for (output) text generation. For a comprehensive introduction to generation
# parameters, refer to https://huggingface.co/blog/how-to-generate
parameters = {
    "max_length": 128,      # Restrict the length of the generated text
    # "num_return_sequences": 5,  # We will inspect several model outputs
    "num_beams": 10         # Use beam search
}


In [ ]:
# Helper functions for running inference queries
def query_endpoint_with_json_payload(payload, endpoint_name):
    encoded_json = json.dumps(payload).encode("utf-8")
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


In [ ]:
def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_texts"]
    return generated_text

In [ ]:
text = """The schema of the table is as below in {} brackets.
              {"table": "au_rmx",
               "table_description": "Table containing billing, customer, material, and sales information, including details such as billing document number, billing date, customer code and name, material description, quantity ordered, plant manufactured, sales value, freight details, account manager, sales office information, and important dates like delivery and order creation.",
              "columns": [
                              ["billing_document": "billing /invoice document number"],
                               ["billing_date": "date when the order was billed"],
                               ["ship_to": "customer code"],
                               ["ship_to_name": "customer name"],
                               ["ship_to_region": "customer region"],
                               ["material": "material brought by the customer]",
                               ["quantity": "quantity ordered"],
                               ["plant": "plant manufactured"],
                               ["plant_name": "plant name"],
                               ["plant_region": "plant region"],
                               ["sales_net_value": "sales value of the material"],
                               ["freight_net_value": "freight value of the material"],
                               ["freight_revenue": "freight revenue of the material"],
                               ["material_description": "material description"],
                               ["cost": "cost of the material"],
                               ["account_manager": "account manager"],
                               ["sales_office": "sales office id"],
                               ["sales_office_desc": "sales office name"],
                               ["del_date": "delivery date"],
                               ["ord_create": "order date"]
                        ]
              }
              Question 1: What is the total sales value for the orders billed on January 6, 2025?
              Answer 1: SELECT SUM(sales_net_value) AS total_sales_value FROM au_rmx WHERE billing_date = '2025-01-06';
              Question 2: Which orders have a delivery date in the year 2025?
              Answer 2: SELECT * FROM au_rmx WHERE strftime('%Y', del_date) = '2025'; 
              Question 3: Which orders have a billing date in January 2025 and a delivery date in February 2025? 
              Answer 3: SELECT * FROM au_rmx WHERE strftime('%Y', billing_date) = '2025' AND strftime('%m', billing_date) = '01' AND strftime('%Y', del_date) = '2025' AND strftime('%m', del_date) = '02'; 
              Question 4: Which sales office had the highest total sales value? 
              Answer 4: SELECT sales_office_desc FROM au_rmx GROUP BY sales_office_desc ORDER BY SUM(sales_net_value) DESC LIMIT 1; 
              Question 5: What is the total quantity of material ordered by each customer? 
              Answer 5: SELECT ship_to_name, SUM(quantity) AS total_quantity_ordered FROM au_rmx GROUP BY ship_to_name;
              Question 6: How many orders have a delivery date before their order date or same as order? 
              Answer 6: SELECT COUNT(*) AS order_count FROM au_rmx WHERE del_date [less than equal to] ord_create;
             Question 7: How many orders have a delivery date before their order date? 
              Answer 7: SELECT COUNT(*) AS order_count FROM au_rmx WHERE del_date [less than] ord_create;
              In a Similar way answer generate a SQL query to 'What is the average sales net value for orders delivered in each region?'"""

In [ ]:
prompt="{context}"
def generate_questions(endpoint_name, text):
    expanded_prompt = prompt.replace("{context}", text)
    payload = {"text_inputs": expanded_prompt, **parameters}
    query_response = query_endpoint_with_json_payload(payload, endpoint_name=endpoint_name)
    generated_texts = parse_response_multiple_texts(query_response)
#     for i, generated_text in enumerate(generated_texts):
#         print(f"Response {i}: {generated_text}{newline}")
    return generated_texts

In [ ]:
x=generate_questions(fine_tuned_name, text)[0]
print(x)

# pred 330 data points

In [ ]:
read_data['pred_sql']=''

In [ ]:
for i in range(0,len(read_data)):
    print(i)
    x=generate_questions(fine_tuned_name, read_data['context'][i])
    read_data['pred_sql'][i]=x[0]
    print(x[0])

In [ ]:
read_data.head()

In [ ]:
csv_buffer = StringIO()
read_data.to_csv(csv_buffer,index=False)
# Upload the CSV string to S3
s3_client.put_object(Body=csv_buffer.getvalue(), Bucket=bucket_name, Key='read_data.csv')

In [ ]:
# Delete resources
pre_trained_predictor.delete_model()
pre_trained_predictor.delete_endpoint()
fine_tuned_predictor.delete_model()
fine_tuned_predictor.delete_endpoint()